# 점수 : 12.852417569

In [4]:
import pandas as pd
import numpy as np
import os
import random
from xgboost import XGBRegressor
import warnings

warnings.filterwarnings(action='ignore')

def seed_everything(seed):
    """모든 난수 생성기의 시드를 고정하여 재현성을 보장"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

# 시드 고정
seed_everything(42)

# 1. 데이터 불러오기
train_df = pd.read_csv('dataset\train.csv', encoding='utf-8')
test_df  = pd.read_csv('dataset\test.csv',  encoding='utf-8')
building_info = pd.read_csv('building_info.csv', encoding='utf-8')

# 2. 건물 정보 정리
building_info = building_info.replace('-', '0')  # 숫자 변환 전 안전 처리
for c in ['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']:
    if c in building_info.columns:
        building_info[c] = building_info[c].astype(float)

# 건물유형 원-핫 인코딩
if '건물유형' in building_info.columns:
    dummies = pd.get_dummies(building_info['건물유형'], prefix='건물유형')
    building_info = pd.concat([building_info.drop(columns=['건물유형']),
                               dummies], axis=1)

# 3. 데이터 병합
train_df = pd.merge(train_df, building_info, on='건물번호', how='left')
test_df  = pd.merge(test_df,  building_info, on='건물번호', how='left')

# 4. 피처 엔지니어링
# '일시' 파싱
train_df['일시'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')
test_df['일시']  = pd.to_datetime(test_df['일시'],  format='%Y%m%d %H')

def add_time_weather_features(df: pd.DataFrame) -> pd.DataFrame:
    # 시간 파생
    df['month'] = df['일시'].dt.month
    df['day'] = df['일시'].dt.day
    df['hour'] = df['일시'].dt.hour
    df['dayofweek'] = df['일시'].dt.dayofweek  # 0=월, 6=일

    # 주기형 인코딩
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)

    # WCI(체감온도) - 풍속(m/s) 기준, km/h로 변환하여 계산
    temp_col = '기온(°C)' if '기온(°C)' in df.columns else None
    hum_col  = '습도(%)' if '습도(%)' in df.columns else None
    wind_col = '풍속(m/s)' if '풍속(m/s)' in df.columns else ('풍속' if '풍속' in df.columns else None)

    if (temp_col is not None) and (wind_col is not None):
        v_kmh = df[wind_col] * 3.6  # 만약 이미 km/h라면 이 변환 제거
        v_pow = np.power(v_kmh, 0.16)
        wci_raw = 13.12 + 0.6215 * df[temp_col] - 11.37 * v_pow + 0.3965 * df[temp_col] * v_pow
        cond = (df[temp_col] <= 10) & (v_kmh >= 4.8)
        df['WCI'] = np.where(cond, wci_raw, df[temp_col])
    else:
        # 풍속/기온이 없으면 WCI는 기온으로 대체
        df['WCI'] = df[temp_col] if temp_col is not None else 0.0

    # THI / CDH / HDH
    if (temp_col is not None) and (hum_col is not None):
        # THI: T - 0.55*(1 - RH)*(T - 14.5), RH=습도(%) / 100
        df['THI'] = df[temp_col] - 0.55 * (1 - (df[hum_col] / 100.0)) * (df[temp_col] - 14.5)
        df['CDH'] = np.maximum(0, df['THI'] - 24)  # Cooling Degree Hours
        df['HDH'] = np.maximum(0, 18 - df['THI'])  # Heating Degree Hours
    else:
        df['THI'] = 0.0
        df['CDH'] = 0.0
        df['HDH'] = 0.0

    return df

train_df = add_time_weather_features(train_df)
test_df  = add_time_weather_features(test_df)

# 5. 학습/예측용 데이터셋 구성
# (THI, CDH, HDH, WCI는 사용하므로 드롭하지 않음)
drop_cols_common = ['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)']
train_y = train_df['전력소비량(kWh)']
train_x = train_df.drop(columns=drop_cols_common + ['전력소비량(kWh)'], errors='ignore')
test_x  = test_df.drop(columns=drop_cols_common, errors='ignore')

# 열 정렬/동기화
test_x = test_x.reindex(columns=train_x.columns, fill_value=0)

# 6. 모델 학습 및 예측
model = XGBRegressor(random_state=42, n_estimators=500, learning_rate=0.05,
                     max_depth=6, subsample=0.8, colsample_bytree=0.8, n_jobs=-1)
model.fit(train_x, train_y)

preds = model.predict(test_x)

# 7. 제출 파일 생성
submission = pd.read_csv('\dataset\building_info.csvsample_submission.csv', encoding='utf-8')
submission['answer'] = preds
submission.to_csv('submission.csv', index=False)

print("모델 학습 및 예측 완료: submission.csv 생성")


OSError: [Errno 22] Invalid argument: 'dataset\train.csv'